In [1]:
# !pip install -U torchdata
# !pip install -U spacy
# !python -m spacy download en_core_web_lg
# !python -m spacy download de_core_news_lg
# !pip install portalocker
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.1 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [2]:
#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

2024-04-25 09:08:38.778263: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 09:08:38.778386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 09:08:38.917310: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using GPU: Tesla P100-PCIE-16GB


In [3]:
PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=8
T5_model = "t5-base"
# T5_model="/kaggle/input/nlpprojt5/T5/checkpoint-1710"
sep='[SEP]'

In [4]:
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from torch.utils.data import DataLoader,Dataset
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch
# from sentence_transformers import SentenceTransformer,util
# import json
# from langchain.document_loaders import CSVLoader,TextLoader
# from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# from langchain.vectorstores import FAISS
# import pandas as pd
# from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# from langchain.embeddings import HuggingFaceInstructEmbeddings
# from InstructorEmbedding import INSTRUCTOR
# from tqdm.notebook import tqdm
# embedding = SentenceTransformerEmbeddings(model_name="hkunlp/instructor-xl",model_kwargs={"device":"cuda"})

In [6]:
# def final_creator(file):
#     final_dict={'id':[],'postText':[],'targetTitle':[],'targetDescription':[],"exctractedParagraph":[]}
#     data=pd.read_json(path+file,lines=True)
#     progress=tqdm(range(len(data)))
#     for i in progress:
#         q=data.iloc[i]
#         l=q['postText'][0]
#         a=q['targetParagraphs']
#         target_title=q['targetTitle']
#         a="\n".join(a)
#         with open("a.txt","w") as f:
#             f.write(a)
#         loader = TextLoader("a.txt")
#         documents = loader.load()
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=100)
#         texts = text_splitter.split_documents(documents)
#         db_embeddings = FAISS.from_documents(texts,embedding)
#         retriever = db_embeddings.as_retriever(search_kwargs={"k":2})
#         top_triples = retriever.get_relevant_documents(l)
#         context=""
#         for i in top_triples:
#             context+=i.page_content
#         final_dict['id'].append(q['uuid'])
#         final_dict['postText'].append(l)
#         final_dict['targetTitle'].append(target_title)
#         final_dict['targetDescription'].append(q['targetDescription'])
#         final_dict['exctractedParagraph'].append(context)
#     final_data=pd.DataFrame(final_dict)
#     final_data.to_json("retriver/final_data_"+file.split(".")[0]+".json")

In [7]:
# train_file="train.jsonl"
# val_file="validation.jsonl"
# test_file="test.jsonl"
# final_creator(train_file)
# final_creator(val_file)
# final_creator(test_file)

In [8]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
def remove_escape(l):
    return l.replace("\n",". ")

In [10]:
def get_label(x):
    return " ".join(x)

In [11]:
train_data=pd.read_json("/kaggle/input/retriever/final_data_train.json")
val_data=pd.read_json("/kaggle/input/retriever/final_data_validation.json")
test_data=pd.read_json("/kaggle/input/retriever/final_data_test.json")

train_data = train_data.replace({None: ''})
val_data = val_data.replace({None: ''})
test_data = test_data.replace({None: ''})

train_real=pd.read_json("/kaggle/input/webis-clickbait-22/train.jsonl",lines=True)
val_real=pd.read_json("/kaggle/input/webis-clickbait-22/validation.jsonl",lines=True)
test_real=pd.read_json("/kaggle/input/webis-clickbait-22/test.jsonl",lines=True)

train_labels=train_real['spoiler'].apply(get_label)
val_labels=val_real['spoiler'].apply(get_label)
test_labels=test_real['spoiler'].apply(get_label)

train_tags=train_real['tags'].apply(get_label)
val_tags=val_real['tags'].apply(get_label)
test_tags=test_real['tags'].apply(get_label)
# train_type=train_real['spoiler']
train_data['exctractedParagraph']=train_data['exctractedParagraph'].apply(remove_escape)
val_data['exctractedParagraph']=val_data['exctractedParagraph'].apply(remove_escape)
test_data['exctractedParagraph']=test_data['exctractedParagraph'].apply(remove_escape)

train_data['context']=train_data['targetTitle'] + ' ' + train_data['targetDescription']+" "+train_data['exctractedParagraph']
val_data['context']=val_data['targetTitle'] + ' ' + val_data['targetDescription']+" "+val_data['exctractedParagraph']
test_data['context']=test_data['targetTitle'] + ' ' + test_data['targetDescription']+" "+test_data['exctractedParagraph']

train_data['question']=train_data['postText']+"?"
val_data['question']=val_data['postText']+'?'
test_data['question']=test_data['postText']+"?"

train_data['spoiler']=train_labels
val_data['spoiler']=val_labels
test_data['spoiler']=test_labels

train_data['tags']=train_tags
val_data['tags']=val_tags
test_data['tags']=test_tags

train_data=train_data[train_data['tags']=='multi']
val_data=val_data[val_data['tags']=='multi']
test_data=test_data[test_data['tags']=='multi']

train_data=train_data[['question','context','spoiler']]

val_data=val_data[['question','context','spoiler']]
test_data=test_data[['question','context','spoiler']]


In [12]:
train_data

,question,context,spoiler
3,Passion is overrated — 7 work habits you need ...,"‘Follow your passion’ is wrong, here are 7 hab...",Purpose connects us to something bigger and in...
9,Hot Sauce Taste Test: Find out which we named ...,"Taste test: 29 hot sauce bottles, ranked From ...",Sriracha Hot Chili Sauce Frank's RedHot Origin...
20,Human remains found in the search for missing ...,Human Remains Found In Search For Missing Teen...,Southampton County Anjelica 'AJ' Hadsell
43,Six lessons from the godfather of California c...,6 Lessons From The Pioneer Of Modern Californi...,Daniel Patterson 1) Eat your veggies. 2) Enjoy...
44,The Fastest Growing Economy in the World! Numb...,The Fastest Growing Economies in the World (No...,1. India 2. Bangladesh 3. China 4. Indonesia 5...
...,...,...,...
3188,Medical marijuana bills advance in these South...,"Medical Marijuana Bills Advance In Georgia, Ke...",Georgia Kentucky
3190,This Guy Cheated On 'The Price Is Right' And F...,This Guy Figured Out ‘The Price Is Right’ Algo...,Terry Kniess All of the prices in the show hav...
3191,7 small changes to try this week?,"7 Small Changes To Try This Week ""One of the ...",Relax with rain sounds. Put pressure on period...
3192,"11 of the weirdest foods you should try, accor...","11 of the weirdest foods you should try, accor...",Cream cheese with a bit of strawberry jam on a...


In [13]:
# val_data.to_csv("check.csv")

In [14]:
# t5
tokenizer = AutoTokenizer.from_pretrained(T5_model)
model = AutoModelWithLMHead.from_pretrained(T5_model)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [15]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
train_dataset = Dataset(pa.Table.from_pandas(train_data)).remove_columns(["__index_level_0__"])

val_dataset = Dataset(pa.Table.from_pandas(val_data)).remove_columns(["__index_level_0__"])
test_dataset = Dataset(pa.Table.from_pandas(test_data)).remove_columns(["__index_level_0__"])
train_dataset,val_dataset,test_dataset

(Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 559
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 143
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 174
 }))

In [16]:
def preproc(data):
    q=[]
    c=[]
    a=[]
    # print(len(data))
    # print(len(data['question']))
    # print(len(data['context']))
    # print(len(data['spoiler']))
    for i in range(len(data['question'])):
        q.append(data['question'][i])
        c.append(data['context'][i])
        if c[i]==None:
            c[i]=""
        a.append(str(data['spoiler'][i]))
    model_inputs=tokenizer(q,c,text_target=a,return_tensors='pt',padding=True,truncation=True,max_length=512)
    return model_inputs

In [17]:
# train_data=Dataset.from_pandas(train_df)
tokenized_train=train_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_val=val_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_test=test_dataset.map(preproc,batched=True,batch_size=BATCH)
tokenized_train,tokenized_val,tokenized_test

Map:   0%|          | 0/559 [00:00<?, ? examples/s]

Map:   0%|          | 0/143 [00:00<?, ? examples/s]

Map:   0%|          | 0/174 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 559
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 143
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 174
 }))

In [18]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [19]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [21]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [22]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)

    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))

    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=12,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
#     report_to="wandb",
#     logging_dir='./lolololol'

    # push_to_hub=True
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]



In [25]:
# results=trainer.evaluate()

In [26]:
# calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

In [27]:
trainer.save_model("T5_overall_passage")

In [28]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [29]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.013412591507579901, 'precisions': [0.20336730172795747, 0.05232837253960634, 0.02566788894709272, 0.01555299539170507], 'brevity_penalty': 0.2954316776030752, 'length_ratio': 0.4505889399081653, 'translation_length': 2257, 'reference_length': 5009}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240425_090939-zx1w8u1g
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sandy-universe-60
wandb: ⭐️ View project at https://wandb.ai/ragha/huggingface
wandb: 🚀 View run at https://wandb.ai/ragha/huggingface/runs/zx1w8u1g


{'eval_loss': 12.214547157287598, 'eval_blue': 0.013412591507579901, 'eval_precisions_1': 0.20336730172795747, 'eval_precisions_2': 0.05232837253960634, 'eval_precisions_3': 0.02566788894709272, 'eval_precisions_4': 0.01555299539170507, 'eval_bp': 0.2954316776030752, 'eval_meteor': 0.10502870207107545, 'eval_bertscore_f1': 0.8268550771406327, 'eval_bertscore_p': 0.8356108271527565, 'eval_bertscore_r': 0.8188920541741382, 'eval_runtime': 33.5427, 'eval_samples_per_second': 5.187, 'eval_steps_per_second': 0.656}


In [30]:
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

[0.06008114311910126,
 0.030476580547208254,
 0.019700255903496278,
 0.013412591507579908]

In [31]:
# idx=
# df=test_data
# for idx in range(len(test_data)):
#     q=df.iloc[idx]['question']
#     c=df.iloc[idx]['context']
#     enc=tokenizer(q+" "+c,return_tensors='pt',padding=True,truncation=True,max_length=512)
#     out=model.generate(input_ids=enc['input_ids'].to(device))
#     out=out.cpu()
#     print(idx)
#     for i in range(len(out)):
#         print("predicted_spoiler",tokenizer.decode(out[i],skip_special_tokens=True))
#         print("actual_spoiler",df.iloc[idx]['spoiler'])